In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# CIFAR100 데이터셋을 가져옵시다. 
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("x_train:", len(x_train), "x_test:", len(x_test))

x_train: 50000 x_test: 10000


In [2]:
img_input = keras.Input(shape=(32, 32, 3))

-------

In [ ]:

x = keras.layers.Conv2D(16, 3, activation='relu')(img_input)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)


predictions = keras.layers.Dense(100, activation='softmax')(x)
model = keras.Model(inputs=img_input, outputs=predictions)

model.summary()

우리가 그동안 학습했던 Convolution Neural Network는 대략 위와 같은 model 구조를 가지고 있을 것입니다. 학습해야 할 파라미터 개수가 대략 326K 정도 되는군요.

1 Epoch만 훈련시켜 봅시다. 우리는 성능 최적화를 시도하는 게 아니라 모델 구조를 분석하고 있는 거니까요.

In [ ]:
# 모델 학습!! 
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)    # 1 Epoch만 학습합니다.

--------

좋습니다. 그럼 이제 위 코드에서 model 부분을 VGG16으로 바꿔보겠습니다. 이전 스텝에서 알려드린 Keras VGG16 코드 구현 소스코드로부터 다음 구현 부분을 찾아서 적용해 봅시다. 지금 필요한 것은 Ctrl+C와 Ctrl+V 입니다.

지금부터 해볼 문제는 코드에서 Block마다 구현이 어떻게 되어 있는지 찾는 것입니다. 구분해서 찾아야 할 블록은 다음과 같습니다.

첫 번째 블록 (첫 번째 Max pooling까지)
두 번째 블록 (두 번째 Max pooling까지)
세 번째 블록 (세 번째 Max pooling까지)
네 번째 블록 (네 번째 Max pooling까지)
다섯 번째 블록 (다섯 번째 Max pooling까지)
여섯 번째 블록 (완전 연결 레이어 + softmax까지)
첫 번째 블록의 예시 답안은 다음과 같습니다. Keras의 Functional API 구조가 익숙하시다면 어려움 없이 답을 찾으실 수 있을 것입니다.

In [3]:
# 첫 번째 블록(예시)
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1')(img_input)
x = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

print('첫 번째 블록 OK!!')

Metal device set to: Apple M1 Pro
첫 번째 블록 OK!!


In [4]:
# Q. 두 번째 블록
# Block 2
x = layers.Conv2D(128, (3, 3),
                    activation='relu',
                    padding='same',
                    name='block2_conv1')(x)
x = layers.Conv2D(128, (3, 3),
                    activation='relu',
                    padding='same',
                    name='block2_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

print('두 번째 블록 OK!!')

두 번째 블록 OK!!


In [5]:
# Q. 세 번째 블록
x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1')(x)
x = layers.Conv2D(256, (3, 3),
                    activation='relu',
                    padding='same',
                    name='block3_conv2')(x)
x = layers.Conv2D(256, (3, 3),
                    activation='relu',
                    padding='same',
                    name='block3_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

print('세 번째 블록 OK!!')

세 번째 블록 OK!!


In [6]:
# Q. 네 번째 블록
x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1')(x)
x = layers.Conv2D(512, (3, 3),
                    activation='relu',
                    padding='same',
                    name='block4_conv2')(x)
x = layers.Conv2D(512, (3, 3),
                    activation='relu',
                    padding='same',
                    name='block4_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

print('네 번째 블록 OK!!')

네 번째 블록 OK!!


In [7]:
# Q. 다섯 번째 블록
x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv1')(x)
x = layers.Conv2D(512, (3, 3),
                    activation='relu',
                    padding='same',
                    name='block5_conv2')(x)
x = layers.Conv2D(512, (3, 3),
                    activation='relu',
                    padding='same',
                    name='block5_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

print('다섯 번째 블록 OK!!')

다섯 번째 블록 OK!!


In [8]:
# Q. 여섯 번째 블록
# [Keras VGG16 코드 구현] 링크의 if include_top: 부분을 유심히 보세요 
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(4096, activation='relu', name='fc1')(x)
x = layers.Dense(4096, activation='relu', name='fc2')(x)

classes=100
x = layers.Dense(classes, activation='softmax', name='predictions')(x)    # CIFAR100을 위한 모델 Output
print('여섯 번째 블록 OK!!')



여섯 번째 블록 OK!!


In [9]:
model = keras.Model(name="VGG-16", inputs=img_input, outputs=x)

model.summary()

Model: "VGG-16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0    